In [2]:
import pandas as pd
import sqlite3
import json

# =========================
# STEP 1: Load orders.csv
# =========================
orders = pd.read_excel("/content/sample_data/orders.xlsx")
print("Orders loaded:", orders.shape)

# =========================
# STEP 2: Load users.json
# =========================
with open("/content/sample_data/users.json", "r") as f:
    users_data = json.load(f)

users = pd.DataFrame(users_data)
print("Users loaded:", users.shape)

# =========================
# STEP 3: Load restaurants.sql
# =========================
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("/content/sample_data/restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)
print("Restaurants loaded:", restaurants.shape)

# =========================
# STEP 4: Merge datasets
# =========================
merged_1 = orders.merge(
    users, on="user_id", how="left"
)

final_df = merged_1.merge(
    restaurants, on="restaurant_id", how="left"
)

print("Final dataset shape:", final_df.shape)

# =========================
# STEP 5: Save final dataset
# =========================
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print(" final_food_delivery_dataset.csv created successfully")


Orders loaded: (10000, 6)
Users loaded: (3000, 4)
Restaurants loaded: (500, 4)
Final dataset shape: (10000, 12)
 final_food_delivery_dataset.csv created successfully


In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter only Gold members
gold_df = df[df["membership"] == "Gold"]

# Group by city and calculate total revenue
city_revenue = gold_df.groupby("city")["total_amount"].sum()

# Display total revenue per city
print(city_revenue)

# Find city with highest revenue
highest_city = city_revenue.idxmax()
highest_revenue = city_revenue.max()

print("\nCity with highest Gold member revenue:")
print(highest_city, "->", highest_revenue)


city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

City with highest Gold member revenue:
Chennai -> 1080909.79


In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Group by cuisine and calculate average order value
avg_order_value = df.groupby("cuisine")["total_amount"].mean()

# Display average order value per cuisine
print(avg_order_value)

# Find cuisine with highest average order value
highest_cuisine = avg_order_value.idxmax()
highest_value = avg_order_value.max()

print("\nCuisine with highest average order value:")
print(highest_cuisine, "->", highest_value)


cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

Cuisine with highest average order value:
Mexican -> 808.0213444401395


In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Group by user and calculate total order value per user
user_total = df.groupby("user_id")["total_amount"].sum()

# Filter users with total orders > 1000
users_above_1000 = user_total[user_total > 1000]

# Count distinct users
count_users = users_above_1000.count()

print("Number of users with total order value > 1000:", count_users)


Number of users with total order value > 1000: 2544


In [6]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Create exact rating ranges as per question
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

# Group by rating range and calculate total revenue
revenue_by_rating = df.groupby("rating_range", observed=True)["total_amount"].sum()

# Print revenue per range
print(revenue_by_rating)

# Find highest revenue range
highest_range = revenue_by_rating.idxmax()
highest_revenue = revenue_by_rating.max()

print("\nRating range with highest total revenue:")
print(highest_range, "->", highest_revenue)


rating_range
3.0–3.5    2136772.70
3.6–4.0    1717494.41
4.1–4.5    1960326.26
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

Rating range with highest total revenue:
4.6–5.0 -> 2197030.75


In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter only Gold members
gold_df = df[df["membership"] == "Gold"]

# Group by city and calculate average order value
avg_order_by_city = gold_df.groupby("city")["total_amount"].mean()

# Print average order value per city
print(avg_order_by_city)

# Find city with highest average order value
highest_city = avg_order_by_city.idxmax()
highest_value = avg_order_by_city.max()

print("\nCity with highest average order value among Gold members:")
print(highest_city, "->", highest_value)


city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64

City with highest average order value among Gold members:
Chennai -> 808.4590800299178


In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Calculate number of distinct restaurants per cuisine
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()

# Calculate total revenue per cuisine
total_revenue = df.groupby("cuisine")["total_amount"].sum()

# Combine both into one DataFrame
summary = pd.DataFrame({
    "distinct_restaurants": restaurant_count,
    "total_revenue": total_revenue
})

# Display summary
print(summary)

# Sort by lowest restaurants but high revenue
summary_sorted = summary.sort_values(
    by=["distinct_restaurants", "total_revenue"],
    ascending=[True, False]
)

print("\nCuisine with lowest restaurants but significant revenue:")
print(summary_sorted.head(1))


         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09

Cuisine with lowest restaurants but significant revenue:
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65


In [9]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Total number of orders
total_orders = len(df)

# Number of orders placed by Gold members
gold_orders = len(df[df["membership"] == "Gold"])

# Calculate percentage
percentage = (gold_orders / total_orders) * 100

# Round to nearest integer
percentage_rounded = round(percentage)
print("Total orders:", total_orders)
print("Gold member orders:", gold_orders)
print("Percentage of orders by Gold members:", percentage_rounded, "%")


Total orders: 10000
Gold member orders: 4987
Percentage of orders by Gold members: 50 %


In [15]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/final_food_delivery_dataset.csv")

# Keep only required combinations
filtered_df = df[
    ((df["membership"] == "Gold") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Gold") & (df["cuisine"] == "Italian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Chinese"))
]

# Group by membership and cuisine, calculate total revenue
revenue_combo = filtered_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum()

print(revenue_combo)

# Find highest revenue combination
highest_combo = revenue_combo.idxmax()
highest_value = revenue_combo.max()

print("\nHighest revenue combination:")
print(highest_combo, "->", highest_value)


membership  cuisine
Gold        Indian      979312.31
            Italian    1005779.05
Regular     Chinese     952790.91
            Indian      992100.27
Name: total_amount, dtype: float64

Highest revenue combination:
('Gold', 'Italian') -> 1005779.05


In [17]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")

# Convert order_date safely (handles mixed formats)
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")

# Extract quarter
df["quarter"] = df["order_date"].dt.quarter

# Calculate total revenue per quarter
quarter_revenue = df.groupby("quarter")["total_amount"].sum()

print(quarter_revenue)

# Highest revenue quarter
print("\nQuarter with highest total revenue:", quarter_revenue.idxmax())


quarter
1.0    1200366.77
2.0    1193374.88
3.0    1239397.48
4.0    1178001.22
Name: total_amount, dtype: float64

Quarter with highest total revenue: 3.0


/tmp/ipython-input-3178628302.py:6: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")


# Numerical Answers:

In [18]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter Gold members
gold_orders = df[df["membership"] == "Gold"]

# Count total orders
total_gold_orders = len(gold_orders)

print("Total orders placed by Gold members:", total_gold_orders)


Total orders placed by Gold members: 4987


In [19]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()

print("Total revenue from Hyderabad:", round(hyderabad_revenue))


Total revenue from Hyderabad: 1889367


In [20]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

distinct_users = df["user_id"].nunique()

print("Number of distinct users:", distinct_users)


Number of distinct users: 2883


In [21]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

gold_avg_order = df[df["membership"] == "Gold"]["total_amount"].mean()

print("Average order value for Gold members:", round(gold_avg_order, 2))


Average order value for Gold members: 797.15


In [22]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter orders with rating >= 4.5
high_rating_orders = df[df["rating"] >= 4.5]

# Count number of such orders
count_orders = len(high_rating_orders)

print("Number of orders placed for restaurants with rating ≥ 4.5:", count_orders)


Number of orders placed for restaurants with rating ≥ 4.5: 3374


In [23]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter only Gold members
gold_df = df[df["membership"] == "Gold"]

# Find total revenue per city (Gold members only)
city_revenue = gold_df.groupby("city")["total_amount"].sum()

# Identify top revenue city
top_city = city_revenue.idxmax()

print("Top revenue city among Gold members:", top_city)

# Count number of orders in that city (Gold only)
orders_count = len(gold_df[gold_df["city"] == top_city])

print("Number of orders in top revenue city (Gold members only):", orders_count)


Top revenue city among Gold members: Chennai
Number of orders in top revenue city (Gold members only): 1337


# Fill in the blanks

In [24]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")
print("Total rows:", df.shape[0])


Total rows: 10000
